In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dataneeds as need
from dataneeds.engine.dask_bag import DaskBagEngine

In [3]:
%run dataneeds/tests/graph.py

In [4]:
with need.request(Node()) as N:
    N.id, N.label#, N.edges.id
rs = N.resolve()
rs

[(dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[0] >> Node().id:Integer,
  dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[1] >> Node().label:String),
 (dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Cons[0] >> Both >> Node().id:Integer,
  dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Cons[1] >> Both >> Node().label:String)]

In [5]:
a,b = rs
a

(dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[0] >> Node().id:Integer,
 dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[1] >> Node().label:String)

In [6]:
a[0].input

Cons(Node().id:Integer, Node().label:String, Sep(',', -1)>>Each)[0]

In [7]:
{'files': {'sep': {'cons': {'Node.label': {}, 'Node.id': {}}}}}

{'files': {'sep': {'cons': {'Node.id': {}, 'Node.label': {}}}}}

In [8]:
with need.request(Edge()) as E:
    E.source.label, E.weight, E.target.id
rs = E.resolve()
a, = rs

In [9]:
a

(dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Cons[1] >> Both >> Edge.source->Node().label,
 dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Cons[2] >> Sep(',', -1) >> Each >> Sep(':', -1) >> Cons >> Cons[1] >> Edge().weight:Floating,
 dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Cons[2] >> Sep(',', -1) >> Each >> Sep(':', -1) >> Cons >> Cons[2] >> Edge.target->Node().id)

In [10]:
e = DaskBagEngine()
e.resolve(a).compute()

[('A', 0.3, 1), ('A', 0.2, 2), ('B', 0.2, 0), ('B', 1.0, 1), ('B', 0.4, 2)]

In [38]:
e.items[1].compute()

[1, 2, 0, 1, 2]

In [15]:
bag = DaskBagEngine().resolve(a)
bag

In [16]:
from dask.async import get_sync

In [17]:
bag.compute(get=get_sync)

[('A', 1), ('A', 2), ('B', 0), ('B', 1), ('B', 2)]

```
\. tests/*.nef
   >> Sep(',')
      >> Cons
         >> Node.label
         >> Each
            >> Sep('.')
               >> Cons
                  >> Edge.id
                  >> Edge.weight 
|> from_files
   |> map(str.sep, sep=',')
      |> l= pluck(1)                
      |> pluck(2)
                         # explode
                         db.zip(l, _.map(len)).map(lambda x: [x[0]]*x[1]) => shape it
         |>                         .concat()
            |> map(str.sep, sep='.')
               |>                            .pluck(1)
               |>                            .pluck(3)
```

In [18]:
import dask.bag as db

In [19]:
files = db.from_filenames('dataneeds/tests/*.nef').map(str.strip)
sep = files.map(lambda x: x.split(',', 2))

labels = sep.pluck(1)

rest = sep.pluck(2)
lst = rest.map(lambda x: x.split(',') if x else [])
weights = lst.concat().map(lambda ls: ls.split(':')).pluck(1)
weights.compute()

explode = db.zip(lst.map(len), labels).map(lambda a,b: a * [b]).concat()
rq = db.zip(explode, weights)

In [100]:
rq.compute()

[('A', '.3'), ('A', '.2'), ('B', '.2'), ('B', '1.'), ('B', '.4')]

In [101]:
lst.compute()

[['0:.3:1', '1:.2:2'], ['2:.2:0', '3:1.:1', '4:.4:2'], []]